In [276]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [277]:
%%microblaze base.PMODA
#include <i2c.h>
#include <stdint.h>
#include <pmod_grove.h>

static i2c device;
static uint8_t device_addr = 0x77;
static uint8_t CCS_addr = 0x5B;
#define DIG_T1_LSB 0x88
#define DIG_T1_MSB 0x89
#define DIG_T2_LSB 0x8A
#define DIG_T2_MSB 0x8B
#define DIG_T3_LSB 0x8C
#define DIG_T3_MSB 0x8D

#define DIG_P1_LSB 0x8E
#define DIG_P1_MSB 0x8F
#define DIG_P2_LSB 0x90
#define DIG_P2_MSB 0x91
#define DIG_P3_LSB 0x92
#define DIG_P3_MSB 0x93
#define DIG_P4_LSB 0x94
#define DIG_P4_MSB 0x95
#define DIG_P5_LSB 0x96
#define DIG_P5_MSB 0x97
#define DIG_P6_LSB 0x98
#define DIG_P6_MSB 0x99
#define DIG_P7_LSB 0x9a
#define DIG_P7_MSB 0x9b
#define DIG_P8_LSB 0x9c
#define DIG_P8_MSB 0x9d
#define DIG_P9_LSB 0x9e
#define DIG_P9_MSB 0x9f

//consult section 4.2.2 of BOSCH BME280 datasheet for parameter readout details
#define DIG_H1_REG 0xA1 
#define DIG_H2_MSB 0xE1
#define DIG_H2_LSB 0xE2
#define DIG_H3_REG 0xE3
#define DIG_H4_FREG 0xE4
#define DIG_H4_SREG 0xE5
#define DIG_H5_FREG 0xE5
#define DIG_H5_SREG 0xE6
#define DIG_H6_REG 0xE7

#define BME280_CONFIG_REG 0xF5//[7:5] standby, delay between samples [4:2] IIR filter settings [1:0] SPI enable



uint16_t dig_T1;
int16_t dig_T2;
int16_t dig_T3;

uint16_t dig_P1;
int16_t dig_P2;
int16_t dig_P3;
int16_t dig_P4;
int16_t dig_P5;
int16_t dig_P6;
int16_t dig_P7;
int16_t dig_P8;
int16_t dig_P9;

uint8_t dig_H1;
int16_t dig_H2;
uint8_t dig_H3;
int16_t dig_H4;
int16_t dig_H5;
int8_t dig_H6;


int32_t t_fine;

uint8_t burst_data[8] = {0};


int copy_params(void* data, int len) {
    int* int_data = (int*)data;
    int_data[1] = dig_H1;
    int_data[2] = dig_H2;
    int_data[3] = dig_H3;
    int_data[4] = dig_H4;
    int_data[5] = dig_H5;
    int_data[6] = dig_H6;

    return len;
}

int ret_tfine() {
    return t_fine;
}

short _read_reg_of_device(uint8_t addr, uint8_t dev_addr){
    uint8_t address_pointer[] = {addr};
    uint8_t data[1];
    i2c_write(device, dev_addr, address_pointer, 1);
    i2c_read(device, dev_addr, data, 1);
    return (int16_t)data[0];
}

short _write_reg_of_device(uint8_t write_addr, uint8_t write_data, uint8_t dev_addr){
    uint8_t addr_join_data[] = {write_addr, write_data};
    i2c_write(device, dev_addr, addr_join_data, 2);
    return 1;
}

int16_t _read_reg_8bit_(uint8_t addr){
    uint8_t address_pointer[] = {addr};
    uint8_t data[1];
    i2c_write(device, device_addr, address_pointer, 1);
    i2c_read(device, device_addr, data, 1);
    return (int16_t)data[0];
}



int16_t _read_register_8bit(uint8_t addr){
    uint8_t address_pointer[] = {addr};
    uint8_t data[1];
    i2c_write(device, device_addr, address_pointer, 1);
    i2c_read(device, device_addr, data, 1);
    return (int16_t)data[0];
}

int _retrieve_temperature_params(){
    dig_T1 = (uint16_t)(_read_register_8bit(DIG_T1_MSB) << 8 | _read_register_8bit(DIG_T1_LSB));
    dig_T2 = (_read_register_8bit(DIG_T2_MSB) << 8 | _read_register_8bit(DIG_T2_LSB));
    dig_T3 = (_read_register_8bit(DIG_T3_MSB) << 8 | _read_register_8bit(DIG_T3_LSB));
    return 0;
}

int _retrieve_pressure_params(){
    dig_P1 = (uint16_t)(_read_register_8bit(DIG_P1_MSB) << 8 | _read_register_8bit(DIG_P1_LSB));
    dig_P2 = (_read_register_8bit(DIG_P2_MSB) << 8 | _read_register_8bit(DIG_P2_LSB));
    dig_P3 = (_read_register_8bit(DIG_P3_MSB) << 8 | _read_register_8bit(DIG_P3_LSB));
    dig_P4 = (_read_register_8bit(DIG_P4_MSB) << 8 | _read_register_8bit(DIG_P4_LSB));
    dig_P5 = (_read_register_8bit(DIG_P5_MSB) << 8 | _read_register_8bit(DIG_P5_LSB));
    dig_P6 = (_read_register_8bit(DIG_P6_MSB) << 8 | _read_register_8bit(DIG_P6_LSB));
    dig_P7 = (_read_register_8bit(DIG_P7_MSB) << 8 | _read_register_8bit(DIG_P7_LSB));
    dig_P8 = (_read_register_8bit(DIG_P8_MSB) << 8 | _read_register_8bit(DIG_P8_LSB));
    dig_P9 = (_read_register_8bit(DIG_P9_MSB) << 8 | _read_register_8bit(DIG_P9_LSB));
    return 0;
}

int _retrieve_humidity_params(){
    dig_H1 = (uint8_t)(_read_register_8bit(DIG_H1_REG));
    dig_H2 = (_read_register_8bit(DIG_H2_MSB) << 8 | _read_register_8bit(DIG_H2_LSB));
    dig_H3 = (uint8_t)(_read_register_8bit(DIG_H3_REG));
    dig_H4 = (_read_register_8bit(DIG_H4_FREG) << 4 | (_read_register_8bit(DIG_H4_SREG) & 0x0F)) & 0x0FFF;
    dig_H5 = (((_read_register_8bit(DIG_H5_FREG) & 0xF0) >> 4) | (_read_register_8bit(DIG_H5_SREG) << 4)) & 0x0FFF;
    dig_H6 = (int8_t)(_read_register_8bit(DIG_H6_REG));
    return 0;
}

short query(uint8_t addr){
    uint8_t address_pointer[] = {addr};
    uint8_t data[1];
    i2c_write(device, CCS_addr, address_pointer, 1);
    i2c_read(device, CCS_addr, data, 1);
    return (int16_t)data[0];
}



int init_BME280(uint8_t addr){
    device = i2c_open(PMOD_G4_B, PMOD_G4_A);
    device_addr = addr;
    uint8_t model_num = _read_register_8bit(0xD0);
    if (model_num != 0x60){
        return -1;
    }
    _retrieve_temperature_params();
    _retrieve_pressure_params();
    _retrieve_humidity_params();
    return (int32_t)model_num;
}

int init_CCS811(uint8_t addr){
    device = i2c_open(PMOD_G4_B, PMOD_G4_A);
    CCS_addr = addr;
    uint8_t model_num = _read_reg_of_device(0x20, CCS_addr);
    if (model_num != 0x81){
        return -1;
    }
    return (int32_t)model_num;
}

int launch_app_CCS811(){
    uint8_t start_command[] = {0xF4};
    i2c_write(device, CCS_addr, start_command, 1);
    return 1;
}

int set_drive_mode_CCS811(uint8_t mode){
    uint8_t drive_mode_settings[] = {0x01, (mode & 0x7c)};
    i2c_write(device, CCS_addr, drive_mode_settings, 2);
    return drive_mode_settings[1];
}

int read_data_CCS811(){
    uint8_t read_data_command[] = {0x02};
    uint8_t data[4];
    i2c_write(device, CCS_addr, read_data_command, 1);
    i2c_read(device, CCS_addr, data, 4);
    return (data[0]<<24 | data[1]<<16 | data[2]<<8 | data[3]);
}

int import_th_params_CCS8110(unsigned short temp, unsigned short hum){
    uint8_t temp_high = (temp >> 8);
    uint8_t temp_low = (temp & 0x00ff);
    uint8_t hum_high = (hum >> 8);
    uint8_t hum_low = (hum & 0x00ff);
    uint8_t write_params_command[] = {0x05, hum_high, hum_low, temp_high, temp_low};
    i2c_write(device, CCS_addr, write_params_command, 5);
    return 0;
}

int compensate_T(){
    int32_t adc_T = ((burst_data[3] << 16 | burst_data[4]<<8 | burst_data[5])>>4);
    int32_t var1 = ((((adc_T>>3) - ((int32_t)dig_T1<<1))) * ((int32_t)dig_T2)) >> 11;
    int32_t  var2 = (((((adc_T>>4) - ((int32_t)dig_T1)) * ((adc_T>>4) - ((int32_t)dig_T1))) >> 12) * ((int32_t)dig_T3)) >> 14;
    t_fine = var1 + var2;
    int32_t T = (t_fine * 5 + 128) >> 8;
    return T;
}

unsigned int compensate_P(){
    int32_t adc_P = ((burst_data[0] << 16 | burst_data[1]<<8 | burst_data[2])>>4);
    int32_t var1, var2;
    uint32_t p;
    var1 = (((int32_t)t_fine)>>1) - (int32_t)64000;
    var2 = (((var1>>2) * (var1>>2)) >> 11 ) * ((int32_t)dig_P6);
    var2 = var2 + ((var1*((int32_t)dig_P5))<<1);
    var2 = (var2>>2)+(((int32_t)dig_P4)<<16);
    var1 = (((dig_P3 * (((var1>>2) * (var1>>2)) >> 13 )) >> 3) + ((((int32_t)dig_P2) * var1)>>1))>>18;
    var1 =((((32768+var1))*((int32_t)dig_P1))>>15);
    if (var1 == 0)
        {
        return 0; // avoid exception caused by division by zero
    }
    p = (((uint32_t)(((int32_t)1048576)-adc_P)-(var2>>12)))*3125;
    if (p < 0x80000000)
        {
        p = (p << 1) / ((uint32_t)var1);
    }
    else
        {
    p = (p / (uint32_t)var1) * 2;
    }
    var1 = (((int32_t)dig_P9) * ((int32_t)(((p>>3) * (p>>3))>>13)))>>12;
    var2 = (((int32_t)(p>>2)) * ((int32_t)dig_P8))>>13;
    p = (uint32_t)((int32_t)p + ((var1 + var2 + dig_P7) >> 4));
    return p;
}

unsigned int compensate_H(){
    int32_t adc_H = ((burst_data[6] << 8) | burst_data[7]);
    int32_t v_x1_u32r;
    v_x1_u32r = (t_fine - ((int32_t)76800));
    v_x1_u32r = (((((adc_H << 14) - (((int32_t)dig_H4) << 20) - (((int32_t)dig_H5) * v_x1_u32r)) +
    ((int32_t)16384)) >> 15) * (((((((v_x1_u32r * ((int32_t)dig_H6)) >> 10) * (((v_x1_u32r *
    ((int32_t)dig_H3)) >> 11) + ((int32_t)32768))) >> 10) + ((int32_t)2097152)) *
    ((int32_t)dig_H2) + 8192) >> 14));
    v_x1_u32r = (v_x1_u32r - (((((v_x1_u32r >> 15) * (v_x1_u32r >> 15)) >> 7) * ((int32_t)dig_H1)) >> 4));
    v_x1_u32r = (v_x1_u32r < 0 ? 0 : v_x1_u32r);
    v_x1_u32r = (v_x1_u32r > 419430400 ? 419430400 : v_x1_u32r);
return (uint32_t)(v_x1_u32r>>12);
}


int burst_read(){
    uint8_t data_addr[1] = {0xF7};
    i2c_write(device, device_addr, data_addr, 1);
    i2c_read(device, device_addr, burst_data, 8);
    return ((burst_data[6] << 8) | burst_data[7]);
}

unsigned int status(){
    uint8_t address_pointer[1] = {0xF3};
    uint8_t data[1] = {0};
    i2c_write(device, device_addr, address_pointer, 1);
    i2c_read(device, device_addr, data, 1);
    return data[0];
}


int set_mode(int mode){
    mode &= 0x3;
    uint8_t address_pointer[1] = {0xF4};
    uint8_t cur_setting[1];
    i2c_write(device, device_addr, address_pointer, 1);
    i2c_read(device, device_addr, cur_setting, 1);
    uint8_t new_setting = (cur_setting[0] & 0xfc) | mode;
    uint8_t addr_joint_setting[2] = {0xF4, new_setting};
    i2c_write(device, device_addr, addr_joint_setting, 2);
    return cur_setting[0];
}


int start_sensing(){
    set_mode(0x00); //sleep mode
    
    
    uint8_t haddr_join_writedata[2] = {0xF2, 0x01}; //turn humidty on x1
    i2c_write(device, device_addr, haddr_join_writedata, 2);
    
    uint8_t taddr_join_writedata[2] = {0xF4, 0x24}; //turn pressure and temp on x1
    i2c_write(device, device_addr, taddr_join_writedata, 2);

    set_mode(0x03);//normal mode
}


int set_standby_BME280(uint8_t mode){
    int32_t cur_mode = set_mode(0x00); //device should be set to sleep mode to ensure configurations will be written
    uint8_t cur_config[1];
    uint8_t config_settings[] = {BME280_CONFIG_REG, 0x00}; //second register will hold new config settings
    i2c_write(device, device_addr, config_settings, 1);
    i2c_read(device, device_addr, cur_config, 1);
    config_settings[1] = ((mode << 5) | (cur_config[0] & 0x1F));
    i2c_write(device, device_addr, config_settings, 2);
    set_mode((uint8_t)cur_mode); //restore mode
    return config_settings[1];
}




In [300]:
class CCS811():
    def __init__(self):
        self.name = 'CCS811'
        self.address = 0x5B
        self.model_number = 0x81
        validation = init_CCS811(self.address)
        if validation != self.model_number:
            print('ERROR: Connection with {} failed. Did not pass initalization test',
                    '(did not retrieve model number 0x{:x})'.format(self.name, self.model_number))
        else:
            print('CCS811 connection valid.')
        self.eC0 = None
        self.eV0 = None
    def start_sensing(self):
        launch_app_CCS811()
        set_drive_mode_CCS811(0x28)

    def set_power_mode(self, mode):
        modes =  {'high': 0x18, 'mid': 0x28, 'low': 0x38}
        try: set_drive_mode_CCS811(modes[mode])
        except KeyError:
            print('Not a valid power mode, valid modes are: {}'.format(', '.join("'{}'".format(x) for x in modes.keys())))
            
    def read_data(self):
        data = read_data_CCS811();
        self.eC0 = data>>16
        self.eV0 = data & 0xff;
        

class BME280():
    def __init__(self):
        self.name = 'BME280'
        self.address = 0x77
        self.model_number = 0x60
        validation = init_BME280(self.address)
        if validation != self.model_number:
            print('ERROR: Connection with BME280 failed. Did not pass initalization test',
                  '(did not retrieve model number 0x{:x})'.format(self.model_number))
        else:
            print('BME280 connection valid.')
        self.temp_C = None
        self.press_Pa = None
        self.hum_per = None
        
    def start(self):
        start_sensing()
    def sleep(self):
        set_mode(0x00)
    def force_sample(self):
        cur_mode = set_mode(0x01) & 0x03
        if cur_mode == 0x03:
            print('sample collected -- passive sensing now disabled. re-enable with start.')
    def retrieve_last_sample(self):
        burst_read()
        self.temp_C  = compensate_T()
        self.press_Pa = compensate_P()
        self.hum_per = compensate_H()
        t_f = (self.temp_C)/100.0 * 9.0/5.0 +32.0
        h_t = self.hum_per/1024.0
        return ((t_f, h_t, self.press_Pa))
    def set_power_mode(self, mode):
        modes =  {'high': 0x00, 'mid': 0x01, 'low': 0x05}
        try: set_standby_BME280(modes[mode])
        except KeyError:
            print('Not a valid power mode, valid modes are: {}'.format(', '.join("'{}'".format(x) for x in modes.keys())))
            
class EnvCombo():
    def __init__(self):
        self.BME280 = BME280()
        self.CCS811 = CCS811()
    
    def start(self):
        self.BME280.start()
        self.CCS811.start_sensing()
        
    def set_power_mode(self, mode):
        self.BME280.set_power_mode(mode)
        self.CCS811.set_power_mode(mode)
        
    def collect_data(self):
        self.BME280.retrieve_last_sample()
        self.CCS811.read_data()
        print(self.BME280.temp_C, self.BME280.hum_per, self.BME280.press_Pa, self.CCS811.eC0, self.CCS811.eV0)
    


In [313]:
EC = EnvCombo()

BME280 connection valid.
CCS811 connection valid.


In [422]:
EC.start()
EC.set_power_mode('low')

In [448]:
EC.collect_data()

2537 56502 85204 657 39


In [217]:
print(hex(set_standby_BME280(0x05)))

print(hex(_read_reg_of_device(0xF5, 0x77)))

0xa0
0xa0


In [38]:
print(burst_read())
t_c = compensate_T()
p_pa = compensate_P()
h_r = compensate_H()
t_f = (t_c/100) * (9/5) + 32

print(p_pa)
print(t_f)
print(h_r/1024.0)

35023
85402
78.386
58.4501953125


In [39]:

#print(hex(set_drive_mode_CCS811(0x08)))
print(hex(_read_reg_of_device(0x00, 0x5B)))
import_th_params_CCS8110((int(((EC.temp_C/100) + 25)*512)),(int(((EC.hum_per/1024.0))*512)))

#print(hex(_write_reg_of_device(0x01, 0x18, 0x5B)))

0x99


0

In [215]:
print(dval>>16)

print(dval & 0xff)

517
17
